In [ ]:
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
from skimage.measure import label, regionprops
import skimage
from skimage.io import imread,imsave
from skimage.filters import threshold_otsu
import numpy as np
import regex as re
from skimage import morphology
import warnings
warnings.filterwarnings('ignore')

import cv2
from scipy.ndimage import filters, measurements


import scipy.ndimage as ndimage
from scipy import ndimage as ndi
from scipy.ndimage.morphology import (
                                    binary_erosion,
                                    binary_dilation, 
                                    binary_fill_holes,
                                    distance_transform_cdt,
                                    distance_transform_edt)
from skimage.feature import peak_local_max

import scipy.io
import re 
import math
from Models import DualEncoding_U_Net,load_model
from skimage.morphology import watershed,remove_small_holes,remove_small_objects,closing,area_closing
from Metrics import get_fast_aji,get_fast_aji_plus,get_fast_pq,get_fast_dice_2,remap_label

# Functions

In [ ]:
def whole_dice_metric(y_pred,y_true):
    smooth = 10e-16
    # single image so just roll it out into a 1D array
    
    m1 =np.reshape(y_pred,(-1))/255
    m2 =np.reshape(y_true,(-1))/255
    
    
    intersection = (m1 * m2)

    score = 2. * (np.sum(intersection) + smooth) / (np.sum(m1) +(np.sum(m2) + smooth))
        
    return score

In [ ]:

def multiple_erosion(img,iter_count=5,selem=morphology.selem.disk(1)):
    for j in (range(iter_count)):
        img=morphology.binary_erosion(img, selem=selem)
    return img

def multiple_dialte(img,iter_count=5,selem=morphology.selem.disk(1)):
    for j in (range(iter_count)):
        img=morphology.binary_dilation(img, selem=selem)
    return img

def coord2array(coord):
    x=[]
    y=[]
    for i in coord:
        x.append(i[0])
        y.append(i[1])
    return (x,y)
def sort_n_array(img):
    img_labels=label(img)
    img_regions=regionprops(img_labels)
    final_gt=np.zeros_like(img)
    for i,region in enumerate(img_regions):
        coordinates=coord2array(list(region.coords))
        final_gt[coordinates]=i+1
        
    return final_gt

In [ ]:
# model=AttnUNet(img_ch=3,output_ch=2,dropout=0.5)


# best post proc for CPM17
def img_recon_CPM17(nuclei,boundary,boundary_length=200,print_prompt=True):
    
    nuclei=nuclei>threshold_otsu(nuclei)
    nuclei=nuclei.astype(np.float32)
    boundary=boundary>50
   
    boundary=boundary.astype(np.float32)
    
    
    nuclei_seeds=nuclei-boundary
    
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    area=len(np.where(nuclei_seeds==1)[0])
    total_area=nuclei_seeds.shape[0]*nuclei_seeds.shape[1]

    if area/total_area>0.15 and area/total_area<0.30:
        nuclei_seeds=multiple_erosion(nuclei_seeds,2)
        iter_count=2
    elif area/total_area>0.30:
        nuclei_seeds=multiple_erosion(nuclei_seeds,3)
        iter_count=3
    elif area/total_area>0.06 and area/total_area<0.15:
        nuclei_seeds=multiple_erosion(nuclei_seeds,1)
        iter_count=2
    else:
#         nuclei_seeds=multiple_erosion(nuclei_seeds,1)
        iter_count=2

    # args:
    #nuclei : raw nuclei image.
    #boundary : raw_boundary predicitons.
    #n_array_gt : GT N-array mask
    #boundary_length : hyperparameter for growing nuclei seeds
    #iter_limt : hyperparameter for growing nuclei seeds
    #print_prompt : wheter or not to show progress bar
    

    labeled_img=label(nuclei_seeds)
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds)
    if print_prompt:
        loop=tqdm(regions)
    else:
        loop=regions
    
    for i,region in enumerate(loop):
        
 
        coordinates=coord2array(list(region.coords))
        x_temp=np.zeros_like(nuclei_seeds)
        x_temp[coordinates]=1
    
        boundary_score=0
        iteration=0
        while boundary_score<boundary_length and iteration<iter_count:
            iteration+=1
            x_temp=morphology.binary_dilation(x_temp, selem=morphology.selem.disk(2)).astype(np.uint8)
            boundary_score=len(np.where(x_temp*boundary==1)[0])



        final_image[np.where(x_temp==1)]=i+1
        final_image=final_image.astype(np.float32)

 
        
    return final_image,area

In [ ]:

def img_recon_CPM17(nuclei,boundary,boundary_length=200,print_prompt=True):
    
    nuclei=nuclei>threshold_otsu(nuclei)
    nuclei=nuclei.astype(np.float32)
    boundary=boundary>50
   
    boundary=boundary.astype(np.float32)
    
    
    nuclei_seeds=nuclei-boundary
    
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    area=len(np.where(nuclei_seeds==1)[0])
    total_area=nuclei_seeds.shape[0]*nuclei_seeds.shape[1]

    if area/total_area>0.15 and area/total_area<0.30:
        nuclei_seeds=multiple_erosion(nuclei_seeds,2)
        iter_count=2
    elif area/total_area>0.30:
        nuclei_seeds=multiple_erosion(nuclei_seeds,3)
        iter_count=3
    elif area/total_area>0.06 and area/total_area<0.15:
        nuclei_seeds=multiple_erosion(nuclei_seeds,1)
        iter_count=2
    else:
#         nuclei_seeds=multiple_erosion(nuclei_seeds,1)
        iter_count=2

    # args:
    #nuclei : raw nuclei image.
    #boundary : raw_boundary predicitons.
    #n_array_gt : GT N-array mask
    #boundary_length : hyperparameter for growing nuclei seeds
    #iter_limt : hyperparameter for growing nuclei seeds
    #print_prompt : wheter or not to show progress bar
    

    labeled_img=label(nuclei_seeds)
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds)
    if print_prompt:
        loop=tqdm(regions)
    else:
        loop=regions
    
    for i,region in enumerate(loop):
        
 
        coordinates=coord2array(list(region.coords))
        x_temp=np.zeros_like(nuclei_seeds)
        x_temp[coordinates]=1
    
        boundary_score=0
        iteration=0
        while boundary_score<boundary_length and iteration<iter_count:
            iteration+=1
            x_temp=morphology.binary_dilation(x_temp, selem=morphology.selem.disk(2)).astype(np.uint8)
            boundary_score=len(np.where(x_temp*boundary==1)[0])



        final_image[np.where(x_temp==1)]=i+1
        final_image=final_image.astype(np.float32)

 
        
    return final_image,area

def watershed_seg(nuclei,boundary):
    
    def gen_inst_dst_map(nuclei):  
        shape = nuclei.shape[:2] # HW
        labeled_img=label(nuclei)
        labeled_img = remove_small_objects(labeled_img, min_size=50)
        regions=regionprops(labeled_img)
    

        canvas = np.zeros(shape, dtype=np.uint8)
        for region in regions:
            coordinates=coord2array(list(region.coords))
            nuc_map=np.zeros(shape)
            nuc_map[coordinates]=1  
            nuc_map=morphology.binary_dilation(nuc_map, selem=morphology.selem.disk(2)).astype(np.uint8)
            nuc_dst = ndi.distance_transform_edt(nuc_map)
            nuc_dst = 255 * (nuc_dst / np.amax(nuc_dst))       
            canvas += nuc_dst.astype('uint8')
        return canvas
    
    nuclei=nuclei>0.45*255#threshold_otsu(nuclei)
    
    
    nuclei=nuclei.astype(np.uint8)
    
    
    nuclei=area_closing(nuclei,20)
    nuclei=closing(nuclei.astype(np.uint8),morphology.selem.square(2))
    nuclei = binary_fill_holes(nuclei)

    nuclei=ndimage.binary_fill_holes(nuclei).astype(int)
    

    
    boundary=boundary>0.3*255#120
    boundary=boundary.astype(np.uint8)
    
    
    nuclei_seeds_ini=nuclei-boundary
    nuclei_seeds_ini[np.where(nuclei_seeds_ini<=0)]=0
    nuclei_seeds_ini[np.where(nuclei_seeds_ini>0)]=1
    
    nuclei_seeds=morphology.binary_erosion(nuclei_seeds_ini, selem=morphology.selem.disk(2)).astype(np.uint8)
    

    labeled_img=label(nuclei_seeds)
    labeled_img = remove_small_objects(labeled_img, min_size=50)
    
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds_ini)
    distance = gen_inst_dst_map(nuclei_seeds_ini)
    markers = ndi.label(nuclei_seeds)[0]
    final_image = watershed(-distance, markers, mask=nuclei,watershed_line=False)
    return final_image,None

In [ ]:

def watershed_seg_kumar(nuclei,boundary):
    
    def gen_inst_dst_map(nuclei):  
        shape = nuclei.shape[:2] # HW
        labeled_img=label(nuclei)
#         labeled_img = remove_small_objects(labeled_img, min_size=30)
        regions=regionprops(labeled_img)
    

        canvas = np.zeros(shape, dtype=np.uint8)
        for region in regions:
            coordinates=coord2array(list(region.coords))
            nuc_map=np.zeros(shape)
            nuc_map[coordinates]=1  
            nuc_map=morphology.binary_dilation(nuc_map, selem=morphology.selem.disk(1)).astype(np.uint8)
            nuc_dst = ndi.distance_transform_edt(nuc_map)
            nuc_dst = 255 * (nuc_dst / np.amax(nuc_dst))       
            canvas += nuc_dst.astype('uint8')
        return canvas
    
    nuclei=nuclei>0.45*255#threshold_otsu(nuclei)
    
    
    nuclei=nuclei.astype(np.uint8)
    
    
    nuclei=area_closing(nuclei,20)
    nuclei=closing(nuclei.astype(np.uint8),morphology.selem.square(2))
    nuclei = binary_fill_holes(nuclei)

    nuclei=ndimage.binary_fill_holes(nuclei).astype(int)
    

    
    boundary=boundary>0.3*255#120
    boundary=boundary.astype(np.uint8)
    
    
    nuclei_seeds_ini=nuclei-boundary
    nuclei_seeds_ini[np.where(nuclei_seeds_ini<=0)]=0
    nuclei_seeds_ini[np.where(nuclei_seeds_ini>0)]=1
    
    nuclei_seeds=morphology.binary_erosion(nuclei_seeds_ini, selem=morphology.selem.disk(1)).astype(np.uint8)
    

    labeled_img=label(nuclei_seeds)
    labeled_img = remove_small_objects(labeled_img, min_size=20)
    
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds_ini)
    distance = gen_inst_dst_map(nuclei_seeds_ini)
    markers = ndi.label(nuclei_seeds)[0]
    final_image = watershed(-distance, markers, mask=nuclei,watershed_line=False)
    return final_image,None

In [ ]:

# def process(pred, model_mode='dcan', ws=True):
#     def gen_inst_dst_map(ann):  
#         shape = ann.shape[:2] # HW
#         nuc_list = list(np.unique(ann))
#         nuc_list.remove(0) # 0 is background

#         canvas = np.zeros(shape, dtype=np.uint8)
#         for nuc_id in nuc_list:
#             nuc_map   = np.copy(ann == nuc_id)    
#             nuc_dst = distance_transform_edt(nuc_map)
#             nuc_dst = 255 * (nuc_dst / np.amax(nuc_dst))       
#             canvas += nuc_dst.astype('uint8')
#         return canvas
    
    
#     assert (pred.shape[2]) == 2, 'Prediction should have contour and blb'
#     blb = pred[...,0]
#     blb = np.squeeze(blb)
#     cnt = pred[...,1]
#     cnt = np.squeeze(cnt)

#     pred = blb - cnt # NOTE
#     pred[pred  > 0.3] = 1 # Kumar 0.3, UHCW 0.3
#     pred[pred <= 0.3] = 0 # CPM2017 0.1
#     pred = measurements.label(pred)[0]
#     pred = remove_small_objects(pred, min_size=20)
    
#     if model_mode != 'dcan':
#         assert len(pred.shape) == 2, 'Prediction shape is not HW'
#         pred[pred  > 0.5] = 1
#         pred[pred <= 0.5] = 0

#         # ! refactor these
#         ws = False if model_mode == 'unet' or model_mode == 'micronet' else ws
#         if ws:
#             dist = measurements.label(pred)[0]
#             dist = gen_inst_dst_map(dist)
#             marker = np.copy(dist)
#             marker[marker <= 125] = 0
#             marker[marker  > 125] = 1
#             marker = binary_fill_holes(marker) 
#             marker = binary_erosion(marker, iterations=1)
#             marker = measurements.label(marker)[0]

#             marker = remove_small_objects(marker, min_size=10)
#             pred = watershed(-dist, marker, mask=pred)
#             pred = remove_small_objects(pred, min_size=10)
#         else:
#             pred = binary_fill_holes(pred) 
#             pred = measurements.label(pred)[0]
#             pred = remove_small_objects(pred, min_size=10)
        
#         if model_mode == 'micronet':
#             # * dilate with same kernel size used for erosion during training
#             kernel = np.array([[0, 1, 0],
#                                [1, 1, 1],
#                                [0, 1, 0]], np.uint8)
    
#             canvas = np.zeros([pred.shape[0], pred.shape[1]])
#             for inst_id in range(1, np.max(pred)+1):
#                 inst_map = np.array(pred == inst_id, dtype=np.uint8)
#                 inst_map = cv2.dilate(inst_map, kernel, iterations=1)
#                 inst_map = binary_fill_holes(inst_map)
#                 canvas[inst_map > 0] = inst_id
#             pred = canvas
#     else:
        
#         canvas = np.zeros([pred.shape[0], pred.shape[1]])

#         k_disk = np.array([
#             [0, 0, 0, 1, 0, 0, 0],
#             [0, 0, 1, 1, 1, 0, 0],
#             [0, 1, 1, 1, 1, 1, 0],
#             [1, 1, 1, 1, 1, 1, 1],
#             [0, 1, 1, 1, 1, 1, 0],
#             [0, 0, 1, 1, 1, 0, 0],
#             [0, 0, 0, 1, 0, 0, 0],
#         ], np.uint8)
#         for inst_id in range(1, np.max(pred)+1):
#             inst_map = np.array(pred == inst_id, dtype=np.uint8)
#             inst_map = cv2.dilate(inst_map, k_disk, iterations=1)
#             inst_map = binary_fill_holes(inst_map)
#             canvas[inst_map > 0] = inst_id
#         pred = canvas
        
#     return pred,None


# CPM 17 Reconstruction by dialation 

In [ ]:
for architecture in ['ab_no_asm','ab_dual_h','attn_unet','DEAU']:
# architecture='attn_unet'
    pred_dir='Results/prediction_{}_CPM_17'.format(architecture)
    img_list=[x for x in os.listdir(pred_dir) if 'nuclei_' in x and 'proc' not in x and 'GT' not in x]
    avg_aji=0
    avg=0
    dq_avg=0
    sq_avg=0
    pq_avg=0
    for img_name in tqdm(img_list):#['nuclei_gbm_image01.png']:


        nuclei=imread(pred_dir+'/'+img_name)    
        boundary=imread((pred_dir+'/'+'bound_'+'_'.join(img_name.split('_')[1:])))

        nuclei_temp=np.expand_dims(nuclei,axis=2)
        boundary_temp=np.expand_dims(boundary,axis=2)

        pred=np.concatenate((nuclei_temp,boundary_temp),axis=2)



        gt=imread('Data/CPM_17/Test/NucleiMaps/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.png')
    #     img_pred,_=process(pred, model_mode='deau', ws=True)
        img_pred,_=watershed_seg(nuclei,boundary)#img_recon(nuclei,boundary,print_prompt=False)#
        img_pred=remap_label(img_pred)
#         plt.imshow(img_pred);plt.show()

        dice_img=(img_pred>0).astype(np.uint8)*255
        dice=whole_dice_metric(dice_img,gt)

        f = open('Data/CPM_17/Test/GT_Mask/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'_mask.txt', 'r')
        x = f.readlines()

        n_array=np.array([int(a) for a in x[1:]]).reshape(list(map(int, re.findall('\d+', x[0]))))
        n_array=remap_label(n_array)
#         plt.imshow(n_array);plt.show()
        imsave(pred_dir+'/'+img_name.split('.')[0]+'_GT.tif',n_array.astype(np.int16))
        img_pred=img_pred.astype(np.int16)
        aji=get_fast_aji(n_array,img_pred)
        pq,_=get_fast_pq(n_array,img_pred)
        dq=pq[0]
        sq=pq[1]
        pq=pq[-1]


    #     imsave(pred_dir+'/'+img_name.split('.')[0]+'_proc.tif',img_pred.astype(np.int16))

        avg+=dice
        avg_aji+=aji
        dq_avg+=dq
        sq_avg+=sq
        pq_avg+=pq
#         print(img_name.split('.')[0],' : ',dice,aji,dq,sq,pq)
    print(architecture)
    print("DICE : ",avg/len(img_list))
    print("AJI : ",avg_aji/len(img_list))
    print("DQ : ",dq_avg/len(img_list))
    print("SQ : ",sq_avg/len(img_list))
    print("PQ : ",pq_avg/len(img_list))


In [ ]:
In this paper, we presented a novel attention-based deep learning approach to segment nuclei. The method makes use of a secondary encoder to constrain the DEAU to focus on only relevant regions of H\&E image such as nuclei. The input to the secondary encoder, stain separated H channel, helps the DEAU to prune the H&E feature space. We conclude that the addition of secondary encoder and the use of logical attention prior, significantly helped the network to learn a better representation in the latent space. This also enabled decoder to produce improved probability maps to segment nuclei.
%We incorporate the novel attention module, we designed a dual encoder U-net (DEAU), in which the secondary Attention Encoding Path (AEP) transforms the attention prior to different feature spaces. 
Quantitative comparison with other contemporary approaches also validates our claim of improved performance through the use of our pseudo hard attention maps. We proved through visual comparison, that the attention maps generated by our method are finer and more accurate than grid-based attention approach.
In the future, we seek to fine-tune the post-processing module to improve instance segmentation performance. 
% Also, we hypothesize to show the capability of proposed architecture on natural images where objects are tiny.

In [ ]:
#DEAU

DICE :  0.8640466507314726
AJI :  0.6524352281589048
DQ :  0.7384281974638034
SQ :  0.7939071636787263
PQ :  0.5875429865357942
    
#attn Unet

DICE :  0.8578644125468057
AJI :  0.636654200951028
DQ :  0.7461005554836503
SQ :  0.7932264655701255
PQ :  0.5933674260220635

In [ ]:
nuclei_hnsc_image08  :  0.8504776970036149 0.5024834437086093 0.6801801801801802 0.7664507726820581 0.5213246246621206
nuclei_lgg_image05  :  0.8849923963429158 0.7637991209927611 0.7777777777777778 0.813689881964702 0.6328699081947683
nuclei_gbm_image01  :  0.6323608345597784 0.3714356867139833 0.5099601593625498 0.6320274094185716 0.3223087984285943
nuclei_gbm_image07  :  0.8794048767047803 0.5825372669099177 0.813953488372093 0.777862511664591 0.6331439048432718
nuclei_lgg_image03  :  0.8673002241656932 0.7175667614134821 0.8163265306122449 0.8009808595047219 0.6538619261263036
nuclei_gbm_image06  :  0.8054084916433094 0.6515418872363667 0.8169014084507042 0.7691383351838418 0.6283101893051102
nuclei_hnsc_image01  :  0.8367801747192013 0.5771474019088016 0.6476190476190476 0.7630900832314125 0.49419167294986716
nuclei_lgg_image04  :  0.8203979115865937 0.5523705075059363 0.7239263803680982 0.698605553165271 0.5057389894079876
nuclei_lgg_image02  :  0.8863404689092762 0.6811495685341491 0.8673835125448028 0.802881276784871 0.6964059820141174
nuclei_lgg_image01  :  0.8817882159044087 0.7057244191057045 0.8412698412698413 0.8145766831066241 0.6852787968992234
nuclei_gbm_image02  :  0.6700441584226201 0.35839274141283217 0.40329218106995884 0.6356426115455397 0.2563496951912053
nuclei_lung_image07  :  0.9285493926805996 0.6675749896620826 0.8229166666666666 0.8184916027417616 0.6735503814229079
nuclei_gbm_image08  :  0.9165822295094047 0.6485727280023542 0.7407407407407407 0.798666238461534 0.5916046210826177
nuclei_lung_image03  :  0.8924826894787635 0.6242109171927219 0.7772277227722773 0.783269040316586 0.6087784125232871
nuclei_lung_image08  :  0.8901537157122894 0.6686207358837796 0.7813267813267813 0.7970811952712801 0.622780884757413
nuclei_lgg_image07  :  0.8358789023482193 0.6124109037488438 0.7346938775510204 0.7640734545042701 0.5613600890235454
nuclei_lung_image02  :  0.8434082036100653 0.6232374592092282 0.7111111111111111 0.7764976325531846 0.5521760942600424
nuclei_gbm_image04  :  0.9040653959358894 0.7740513392857142 0.8941684665226782 0.8186776233336504 0.732035715032681
nuclei_gbm_image03  :  0.8663270129733704 0.7036691720695393 0.85 0.7731208785114023 0.6571527467346919
nuclei_lung_image05  :  0.9108088046562285 0.6456740783559405 0.782258064516129 0.8175804999139482 0.6395589394488143
nuclei_hnsc_image02  :  0.8517018897539361 0.5882586472135378 0.7078651685393258 0.8003408351525043 0.5665334001641322
nuclei_lung_image04  :  0.87306797343319 0.480140562967202 0.6447761194029851 0.7716574940835972 0.4975463245434537
nuclei_hnsc_image07  :  0.8610366011940815 0.6587899010374555 0.8274647887323944 0.7576825771858988 0.6269556536573458
nuclei_hnsc_image06  :  0.835657614655228 0.6622735943878074 0.7774647887323943 0.7642075806412709 0.5941444852309599
nuclei_lung_image06  :  0.9060736693840966 0.6151833378111603 0.7337278106508875 0.8198742263703576 0.6015645211238126
nuclei_hnsc_image05  :  0.859765718383935 0.7021020551691802 0.8 0.7995249677284383 0.6396199741827506
nuclei_lgg_image08  :  0.8689715079916609 0.6902429052768105 0.7816091954022989 0.7769148238645222 0.6072437703768679
nuclei_hnsc_image03  :  0.8847231230888758 0.7457171446827492 0.7920792079207921 0.8427383092988674 0.6675154925139544
nuclei_gbm_image05  :  0.8643800596506178 0.589087635751205 0.7 0.7246950952815278 0.5072865666970694
nuclei_lgg_image06  :  0.8590640201587366 0.6577172827172827 0.7238095238095238 0.8191655847306879 0.5929198518050693
nuclei_lung_image01  :  0.5727587309087372 0.3679041118694856 0.5172413793103449 0.7675145077709914 0.396990262640168
nuclei_hnsc_image04  :  0.8353170024067785 0.6383022078226406 0.7613636363636364 0.8097338834719164 0.6165019340070272
DICE :  0.846127178371153
AJI :  0.619621578611227
DQ :  0.7425136111781029
SQ :  0.7773891884200127
PQ :  0.5807376440390992

# BEST CPM 17 dialation 
nuclei_gbm_image01  :  0.8214666279182408 12868<br/>
nuclei_lgg_image02  :  0.871614730878187 64306<br/>
nuclei_lgg_image04  :  0.8228840588270038 41948<br/>
nuclei_gbm_image05  :  0.8945122519332899 16635<br/>
nuclei_lung_image05  :  0.8967014977589723 52581<br/>
nuclei_lung_image08  :  0.8757263674277889 43220<br/>
nuclei_gbm_image04  :  0.8841680624690368 55356<br/>
nuclei_gbm_image02  :  0.7925804468128715 11195<br/>
nuclei_hnsc_image03  :  0.8868635778021663 58493<br/>
nuclei_hnsc_image04  :  0.8471280383960366 54710<br/>
nuclei_gbm_image08  :  0.9125669596006607 41017<br/>
nuclei_lung_image06  :  0.9118775898376469 44832<br/>
nuclei_lgg_image03  :  0.8690029420020168 31731<br/>
nuclei_lgg_image05  :  0.8358029450312235 21205<br/>
nuclei_lgg_image06  :  0.8289500267395568 29558<br/>
nuclei_gbm_image06  :  0.8605572485851111 14334<br/>
nuclei_gbm_image03  :  0.8747670485157347 44686<br/>
nuclei_lung_image02  :  0.8159096441263813 41334<br/>
nuclei_gbm_image07  :  0.8958297068112302 26885<br/>
nuclei_lung_image03  :  0.8657846436219028 34675<br/>
nuclei_hnsc_image01  :  0.8072174361824606 13508<br/>
nuclei_hnsc_image05  :  0.8295879643401209 42940<br/>
nuclei_lung_image07  :  0.9064625496599206 54217<br/>
nuclei_hnsc_image06  :  0.8209365064071683 59521<br/>
nuclei_lgg_image08  :  0.8726363494295983 39956<br/>
nuclei_lung_image01  :  0.7938195935348232 56722<br/>
nuclei_lgg_image01  :  0.7761772950863026 19420<br/>
nuclei_hnsc_image02  :  0.843762329361635 56059<br/>
nuclei_hnsc_image07  :  0.8488544610039589 46205<br/>
nuclei_lgg_image07  :  0.8883423816373563 19562<br/>
nuclei_lung_image04  :  0.8555564148171062 46814<br/>
nuclei_hnsc_image08  :  0.8295273080200204 63858<br/>
0.8542992188929854<br/>


if area>30000 and area<50000:<br/>
         <p>nuclei_seeds=multiple_erosion(nuclei_seeds,2</p>)<br/>
         <p>iter_count=2</p><br/>
elif area>50000:<br/>
   <p> nuclei_seeds=multiple_erosion(nuclei_seeds,3)</p><br/>
    <p>iter_count=3</p><br/>
else:<br/>
    <p>iter_count=2</p><br/>

# prediction of CoNSep

In [ ]:
for architecture in ['ab_no_asm','ab_dual_h','attn_unet','DEAU']:
    pred_dir_CoNSep='Results/prediction_{}_CoNSeP/'.format(architecture)
    img_list=[x for x in os.listdir(pred_dir_CoNSep) if 'test' and 'nuc' in x and 'proc' not in x]
    avg=0
    avg_aji=0
    dq_avg=0
    sq_avg=0
    pq_avg=0
    for img_name in tqdm(img_list):
        boundary=imread(pred_dir_CoNSep+'/'+'bound_'+'_'.join(img_name.split('_')[1:]))
        nuclei=imread(pred_dir_CoNSep+'/'+img_name)
        gt=np.load('Data/CoNSeP/Test/Labels/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.npy')[:,:,0]
        gt=remap_label(gt)
    #     plt.imshow(gt);plt.show()

    #     imsave(pred_dir_CoNSep+'/'+'GT'+'_'.join(img_name.split('_')[1:]),gt.astype(np.int16))
        gt_dice=(gt>0).astype(np.uint8)*255

        img_pred,_=watershed_seg(nuclei,boundary)#(nuclei,boundary,print_prompt=False)
    #     plt.imshow(img_pred);plt.show()
        img_pred=remap_label(img_pred)
    #     plt.imshow(img_pred);plt.show()
        pred_dice=(img_pred>0).astype(np.uint8)*255

    #     imsave(pred_dir_CoNSep+'/'+img_name.split('.')[0]+'_proc.tif',img_pred.astype(np.int16))

        dice=whole_dice_metric(pred_dice,gt_dice)
        aji=get_fast_aji(gt,img_pred)
        pq,_=get_fast_pq(gt,img_pred)
        dq=pq[0]
        sq=pq[1]
        pq=pq[-1]
    #     df = df.append({'Image': img_name.split('.')[0],'Dice': dice,\
    #                     'AJI': aji,'DQ':dq,'SQ':sq,'PQ':pq}, ignore_index=True)

        avg+=dice
        avg_aji+=aji
        dq_avg+=dq
        sq_avg+=sq
        pq_avg+=pq
    #     print(img_name.split('.')[0],' : ',dice,aji,dq,sq,pq)
    print(architecture)
    print("DICE : ",avg/len(img_list))
    print("AJI : ",avg_aji/len(img_list))
    print("DQ : ",dq_avg/len(img_list))
    print("SQ : ",sq_avg/len(img_list))
    print("PQ : ",pq_avg/len(img_list))
# df.to_csv('metric_comp.csv',index=False)


In [ ]:
# DEAU
DICE :  0.8219939981778751
AJI :  0.4849404251757841
DQ :  0.49200648582495043
SQ :  0.744809133312174
PQ :  0.3680040335169987
    
#attn _unet

DICE :  0.8360142826845708
AJI :  0.5243323093265808
DQ :  0.5460233083395224
SQ :  0.7605303411114803
PQ :  0.416636735150674

In [ ]:

#DEAU
nuclei_test_1  :  0.8463013237873291 0.4182112939116927 0.47538538040775735 0.7248789971636728 0.34459687781624626
nuclei_test_2  :  0.709229509874143 0.4506969976028625 0.40956651718983555 0.7311696202938921 0.2994625948587839
nuclei_test_12  :  0.8291694704139047 0.5376974324036177 0.580952380952381 0.7513935744305595 0.43652388609775367
nuclei_test_6  :  0.8486639867580988 0.5930537122472912 0.6458333333333334 0.7711865086343102 0.498057953492992
nuclei_test_4  :  0.8426357427153457 0.44903473444400055 0.3883817427385892 0.7212577790669021 0.2801233531977678
nuclei_test_7  :  0.7906340087520999 0.4551170535314876 0.494279176201373 0.7059956414248243 0.3489589440452221
nuclei_test_10  :  0.7886985593580937 0.3550592581191319 0.3023740108288213 0.7426290051355944 0.22455171084066702
nuclei_test_8  :  0.8413638356489591 0.3537309081023964 0.34922766957689727 0.7051699601777296 0.24626486184850196
nuclei_test_11  :  0.8328158017840724 0.38748897418170547 0.386317907444668 0.7034032509321679 0.27173727198989184
nuclei_test_3  :  0.8308252529560356 0.45993231116018934 0.42424242424242425 0.7310340531047655 0.3101356588929308
nuclei_test_9  :  0.8127587025107875 0.5055209250590907 0.5831702544031311 0.7533345029853996 0.4393222737566518
nuclei_test_13  :  0.8299910998191005 0.47970291726674363 0.5657120439387995 0.7498332553951031 0.4241897035228476
nuclei_test_5  :  0.7911380703431401 0.4456158588372722 0.5166744950681071 0.7470182885853841 0.3859652970614949
nuclei_test_14  :  0.820207254958369 0.5262447196878716 0.5744400527009222 0.7598641159700936 0.43649638282340025
DICE :  0.8153166156913914
AJI :  0.4583647926110967
DQ :  0.4783255277876458
SQ :  0.7355834680928856
PQ :  0.3533133407317966

# prediction of kumar

In [ ]:
for architecture in ['ab_no_asm','ab_dual_h','attn_unet','DEAU']:

    pred_dir='Results/prediction_{}_kumar'.format(architecture)
    img_list=[x for x in os.listdir(pred_dir) if 'nuc' in x and 'proc' not in x]
    avg=0
    avg_aji=0
    dq_avg=0
    sq_avg=0
    pq_avg=0
    for img_name in tqdm(img_list):
        nuclei=imread(pred_dir+'/'+img_name)


        boundary=imread(pred_dir+'/'+'bound_'+'_'.join(img_name.split('_')[1:]))

        gt=imread('Data/kumar/processed_data/n_array_gt/'+img_name.split('.')[0].split('_')[1:][0]+'.tif')

        img_pred,_=watershed_seg(nuclei,boundary)


    #     imsave(pred_dir+'/'+img_name.split('.')[0]+'_proc.tif',img_pred.astype(np.uint16))
    #     imsave(pred_dir+'/'+img_name.split('.')[0]+'GT.tif',gt.astype(np.uint16))

        img_pred=remap_label(img_pred)
        gt=remap_label(gt)
      

        pred_dice=(img_pred>0).astype(np.uint8)*255
        gt_dice=(gt>0).astype(np.uint8)*255

        dice=whole_dice_metric(pred_dice,gt_dice)
        aji=get_fast_aji(gt,img_pred)
        pq,_=get_fast_pq(gt,img_pred)
        dq=pq[0]
        sq=pq[1]
        pq=pq[-1]


        avg+=dice
        avg_aji+=aji
        dq_avg+=dq
        sq_avg+=sq
        pq_avg+=pq
#         print(img_name.split('.')[0],' : ',dice,aji,dq,sq,pq)
    print(architecture)
    print("DICE : ",avg/len(img_list))
    print("AJI : ",avg_aji/len(img_list))
    print("DQ : ",dq_avg/len(img_list))
    print("SQ : ",sq_avg/len(img_list))
    print("PQ : ",pq_avg/len(img_list))



In [ ]:
DEAU
DICE :  0.8007710395338207
AJI :  0.5242146696686297
DQ :  0.6238247374084731
SQ :  0.7332281499407338
PQ :  0.45950728884165937
Attn -Unet
DICE :  0.7997172426990424
AJI :  0.5218101986884494
DQ :  0.6292659914623709
SQ :  0.7348405345017782
PQ :  0.4647423810147938

In [ ]:
nuclei_TCGA-21-5786-01Z-00-DX1  :  0.7939573920534472 0.4253963528098251 0.4187946884576098 0.6841894635333533 0.2865349132264299
nuclei_TCGA-G2-A2EK-01A-02-TSB  :  0.6771630887333224 0.3688979995758818 0.3952225841476656 0.6654823043532099 0.2630136360310189
nuclei_TCGA-HE-7128-01Z-00-DX1  :  0.7481460395316925 0.4321901586737316 0.5216981132075472 0.6801244497149941 0.35481964216263373
nuclei_TCGA-B0-5710-01Z-00-DX1  :  0.8371678143920137 0.5546784477676666 0.629878869448183 0.7482800256086608 0.47132577656104074
nuclei_TCGA-G9-6348-01Z-00-DX1  :  0.8004003184351188 0.4481540517477338 0.5187713310580204 0.6797116258404847 0.3526149048728794
nuclei_TCGA-RD-A8N9-01A-01-TS1  :  0.8441525889671083 0.3976700370872224 0.3738467709586843 0.7059031328934104 0.2638996068418205
nuclei_TCGA-KB-A93J-01A-01-TS1  :  0.8603375481472607 0.35996803588866244 0.3043175487465181 0.7108346135358878 0.21631944715541992
nuclei_TCGA-21-5784-01Z-00-DX1  :  0.7501050059797298 0.39838413882703855 0.3854922279792746 0.6840204276872224 0.26368455865248364
nuclei_TCGA-E2-A1B5-01Z-00-DX1  :  0.7648538219862876 0.4125644941175277 0.47941176470588237 0.6829604783825349 0.3274192881657447
nuclei_TCGA-AY-A8YK-01A-01-TS1  :  0.6949542526122029 0.27841657279142507 0.26303317535545023 0.6529629536528266 0.17175091908877668
nuclei_TCGA-CH-5767-01Z-00-DX1  :  0.8039830730800379 0.3784872436096356 0.5750350631136045 0.6401555035725635 0.3681118603993703
nuclei_TCGA-DK-A2I6-01A-01-TS1  :  0.8450002481996334 0.5228449069926487 0.5949926362297496 0.739299267919018 0.4398776203818605
nuclei_TCGA-E2-A14V-01Z-00-DX1  :  0.8334076605527561 0.5178882120781318 0.6150178784266984 0.7304434460333293 0.44923577849010476
nuclei_TCGA-NH-A8F7-01A-01-TS1  :  0.7622460494090324 0.3343282491106671 0.36027713625866054 0.6651215082420957 0.23962807225350333

DICE :  0.7868482072914033
AJI :  0.41641920721984277
DQ :  0.4596992705781106
SQ :  0.6906778000692565
PQ :  0.3191597160202205

In [ ]:
import numpy as np
import glob
import os
import xml.etree.ElementTree as ET
import skimage
from skimage.io import imread,imsave
from skimage import draw
import imutils
from skimage.filters import threshold_otsu
from skimage.util import pad
from tqdm import tqdm_notebook as tqdm 



def convert_to_png(tif_dir,png_dir):
    images_name=[x for x in os.listdir(tif_dir) if '.tif' in x]
    
    if not os.path.exists(png_dir):
        os.mkdir(png_dir)
        print("made dir {}".format(png_dir))
    for im in tqdm(images_name):
        img_name=im
        img=imread(os.path.join(tif_dir,im))
        img_name=img_name.replace('.tif','.png')    
        imsave(os.path.join(png_dir,img_name),img)
        
        
def rot_image(img):
    test=img.copy()
    rot=test[:, ::-1]
    rot=imutils.rotate(rot,90)
    return rot

def poly2boundry(x,y,img_array,val):
    if len(x)==2 and len(y)==2:
        rr,cc=draw.line(x[0],y[0],x[1],y[1])
    else:
        rr, cc = draw.polygon_perimeter(x, y)
    img_array[rr,cc]=[val]
    return img_array

def check_in_bounds(x,y,bound):

    if x>=bound:
        x=bound-1
    if y>=bound:
        y=bound-1
    if x<0:
        x=0
    if y<0:
        y=0
    return x,y
        
        
def create_data_vahadane(png_dir,annotation_dir,n_arry_dir):#,nucleus_dir,boundary_dir):
    img_name_list=[x for x in os.listdir(png_dir) if '.png' in x]
#     name=img_name_list[0]
    
   
    if not os.path.exists(n_arry_dir):
        os.mkdir(n_arry_dir)
        print("made dir {}".format(n_arry_dir))


    

    loop=tqdm(img_name_list)
    for name in loop:
        sample_img=imread(os.path.join(png_dir,name))
        r,c,_=sample_img.shape

        xml_name=os.path.join(annotation_dir,name.split('.')[0]+'.xml')

        tree=ET.parse(xml_name)
        root=tree.getroot()
        img_nucleus=np.zeros(shape=(r,c),dtype=np.uint16)

        loop.set_postfix(Regions=len([v.tag for v in root.iter('Vertices')]))

        for i,v in enumerate(root.iter('Vertices')):
            X=[]
            Y=[]

            for child in v:
                x=int(eval(child.attrib['X']))
                y=int(eval(child.attrib['Y']))
                x,y=check_in_bounds(x,y,1000)

                X.append(x)
                Y.append(y)


            r_nucleus,c_nucleus=draw.polygon(X,Y)
            img_nucleus[r_nucleus,c_nucleus]=i+1




#     img_nucleus.dtype=np.uint8


#     img_nucleus=rot_image(img_nucleus)

 

        
        img_nucleus=rot_image(img_nucleus)
        imsave(os.path.join(n_arry_dir,name.split('.')[0]+'.tif'),img_nucleus)
        
#     return img_nucleus
    print("DONE")



In [ ]:
png_dir='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/Data/kumar/png_images'
annotation_dir='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/Data/kumar/Annotations'
n_arry_dir='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/Data/kumar/processed_data/n_array_gt'
create_data_vahadane(png_dir,annotation_dir,n_arry_dir)



In [ ]:
('_'),join(img_name.split('.')[0].split('_')[1:])

TCGA-E2-A14V-01Z-00-DX1  :  0.8225290612209419<br/>
TCGA-21-5786-01Z-00-DX1  :  0.6991805205647318<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7702200475923103<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.7629844577597736<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7624224458621036<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.7970872142252329<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8176449329279959<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.7966429519738157<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8649618223341795<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8129848869951398<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7642849375366202<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.7747906286441217<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7865019409500186<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.6759419369045772<br/>
0.77915555610654<br/>
# Model 30th Jan 2020 model_optim.pth
TCGA-21-5784-01Z-00-DX1  :  0.8116361668301593 108<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8782868692362495 117<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.815197744362616 115<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7743249219687112 114<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7873815363001121 108<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.8273699189706346 112<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8798372531910145 118<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.883796541868124 115<br/>
TCGA-21-5786-01Z-00-DX1  :  0.7315855593758265 111<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.8599896627535161 113<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7949021136999774 113<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.802531826634244 110<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.788953461927371 105<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.828047288316124 109<br/>
0.8188457761024771<br/>

# Model 31st 2 channel pred with only otsu
TCGA-G9-6348-01Z-00-DX1  :  0.7819525407623568 108<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8811115372673644 122<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.8138068067885593 111<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8401957024486049 118<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7201258488319375 124<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.8268603691132442 107<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.7434148978191721 94<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7782766771403891 108<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.7851957232741299 106<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8711696861502439 120<br/>
TCGA-21-5786-01Z-00-DX1  :  0.7051864689168885 107<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.8391341602292577 113<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.6983577090485372 110<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.6546214106691893 126<br/>
0.7813863956042768<br/>
# Model 31st 2 channel pred with boundary constratined growing of nuclei seeds
TCGA-G9-6348-01Z-00-DX1  :  0.7193487672267217 108<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.7978143579195152 122<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.7869126701887118 111<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8429467108874303 118<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7064353205931725 124<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.7426428449341953 107<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.6464007628952234 94<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7619342935549388 108<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.751238314651544 106<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8106484504188964 120<br/>
TCGA-21-5786-01Z-00-DX1  :  0.5648986336791768 107<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.815677097344578 113<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.760678717070453 110<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.6709322769456747 126<br/>
0.7413220870221593<br/>

# predicition CoNSep

In [ ]:
!ls /home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/Data/kumar/processed_data


# Best Post PRocess for CoNSep


def img_recon_consep(nuclei,boundary,print_prompt=True):
    

    nuclei=nuclei>0.6*255
    
    nuclei=nuclei.astype(np.float32)
    nuclei=ndimage.binary_fill_holes(nuclei).astype(int)
    
    boundary=boundary>0.5*255#120
    boundary=boundary.astype(np.float32)
        
    nuclei_seeds=nuclei-boundary
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    nuclei_seeds=ndimage.binary_fill_holes(nuclei_seeds).astype(int)
    
    


    labeled_img=label(nuclei_seeds)
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds)
    if print_prompt:
        loop=tqdm(regions)
    else:
        loop=regions
    
    for i,region in enumerate(loop):
        
        coordinates=coord2array(list(region.coords))
        area_region=len(coordinates[0])
        boundary_length=[math.sqrt(area_region/math.pi)*math.pi*2 if math.sqrt(area_region/math.pi)*math.pi*2>50 else 50][-1]
        x_temp=np.zeros_like(nuclei_seeds)
        x_temp[coordinates]=1

        x_temp=morphology.binary_dilation(x_temp, selem=morphology.selem.disk(2)).astype(np.uint8)
           



        final_image[np.where(x_temp==1)]=i+1
        final_image=final_image.astype(np.float32)

 
        
    return final_image


def watershed_seg_consep(nuclei,boundary):
    
    nuclei_th=(nuclei>0.25*255).astype(np.float32)
    boundary_th=(boundary>0.25*255).astype(np.float32)
   
    
    nuclei_seeds=nuclei_th-boundary_th
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    nuclei_seeds=nuclei_seeds.astype(np.uint8)
    nuclei_seeds=remove_small_holes(nuclei_seeds)
    nuclei_seeds=label(nuclei_seeds)
#     nuclei_seeds=remove_small_objects(nuclei_seeds)
    nuclei_distance_map=ndi.distance_transform_edt(nuclei_seeds)
    
    
    local_maxi = peak_local_max(nuclei_distance_map, indices=False, footprint=np.ones((3, 3)),
                                labels=nuclei_th)
    markers = ndi.label(local_maxi)[0]
    seg_image = watershed(-nuclei_distance_map, markers, mask=nuclei_th)
    
    
    return seg_image

In [ ]:
def img_recon_consep(nuclei,boundary,print_prompt=True):
    
    # args:
    #nuclei : raw nuclei image.
    #boundary : raw_boundary predicitons.
    #print_prompt : wheter or not to show progress bar

    nuclei=nuclei>0.6*255#threshold_otsu(nuclei)
    
    nuclei=nuclei.astype(np.float32)
    nuclei=ndimage.binary_fill_holes(nuclei).astype(int)
    
    boundary=boundary>0.5*255#120
    boundary=boundary.astype(np.float32)
    
    
    nuclei_seeds=nuclei-boundary
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    
    nuclei_seeds=ndimage.binary_fill_holes(nuclei_seeds).astype(int)
    area=len(np.where(nuclei_seeds==1)[0])
    total_area=nuclei_seeds.shape[0]*nuclei_seeds.shape[1]
    
    

    nuclei_seeds=morphology.binary_erosion(nuclei_seeds, selem=morphology.selem.disk(1)).astype(np.uint8)


    labeled_img=label(nuclei_seeds)
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(nuclei_seeds)
    if print_prompt:
        loop=tqdm(regions)
    else:
        loop=regions
    
    for i,region in enumerate(loop):
        
 
        coordinates=coord2array(list(region.coords))
        area_region=len(coordinates[0])

        x_temp=np.zeros_like(nuclei_seeds)
        x_temp[coordinates]=1

        x_temp=morphology.binary_dilation(x_temp, selem=morphology.selem.disk(1)).astype(np.uint8)

        final_image[np.where(x_temp==1)]=i+1
        final_image=final_image.astype(np.float32)

 
        
    return final_image,area/total_area


def watershed_seg_consep(nuclei,boundary):
    
    nuclei_th=(nuclei>0.25*255).astype(np.float32)
    boundary_th=(boundary>0.25*255).astype(np.float32)
   
    
    nuclei_seeds=nuclei_th-boundary_th
    nuclei_seeds[np.where(nuclei_seeds<0)]=0
    nuclei_seeds=nuclei_seeds.astype(np.uint8)
    nuclei_seeds=remove_small_holes(nuclei_seeds)
    nuclei_seeds=label(nuclei_seeds)
#     nuclei_seeds=remove_small_objects(nuclei_seeds)
    nuclei_distance_map=ndi.distance_transform_edt(nuclei_seeds)
    
    
    local_maxi = peak_local_max(nuclei_distance_map, indices=False, footprint=np.ones((3, 3)),
                                labels=nuclei_th)
    markers = ndi.label(local_maxi)[0]
    seg_image = watershed(-nuclei_distance_map, markers, mask=nuclei_th)
    
    
    return seg_image

In [ ]:
avg=0
avg_aji=0
dq_avg=0
sq_avg=0
pq_avg=0
pred_list=[x for x in os.listdir('hovernet/') if 'test' in x]
for pred in tqdm(pred_list):
    gt_name='_'.join(pred.split('_')[-2:])
    gt=np.load('other_data/CoNSeP/Test/Labels/'+gt_name)[:,:,0]
    gt=remap_label(gt)
    gt_dice=(gt!=0).astype(np.uint8)*255
    img_pred=np.load('hovernet/'+pred)
    img_pred=remap_label(img_pred)
    pred_dice=(img_pred!=0).astype(np.uint8)*255
    dice=whole_dice_metric(pred_dice,gt_dice)
    aji=get_fast_aji(gt,img_pred)
    pq,_=get_fast_pq(gt,img_pred)
    dq=pq[0]
    sq=pq[1]
    pq=pq[-1]
    
    avg+=dice
    avg_aji+=aji
    dq_avg+=dq
    sq_avg+=sq
    pq_avg+=pq
    print(pred.split('.')[0],' : ',dice,aji,dq,sq,pq)
print("DICE : ",avg/len(pred_list))
print("AJI : ",avg_aji/len(pred_list))
print("DQ : ",dq_avg/len(pred_list))
print("SQ : ",sq_avg/len(pred_list))
print("PQ : ",pq_avg/len(pred_list))
    

# model 4th feb, predicts only nuclei (CPM-17)

nuclei_gbm_image01  :  0.7277193766051461 108<br/>
nuclei_lgg_image02  :  0.8795810296965031 120<br/>
nuclei_lgg_image04  :  0.8498570690545892 120<br/>
nuclei_gbm_image05  :  0.8793331994376381 115<br/>
nuclei_lung_image05  :  0.9245344129554656 119<br/>
nuclei_lung_image08  :  0.8866071888262075 118<br/>
nuclei_gbm_image04  :  0.901806647358985 119<br/>
nuclei_gbm_image02  :  0.7480424652927895 110<br/>
nuclei_hnsc_image03  :  0.8753627522274856 123<br/>
nuclei_hnsc_image04  :  0.8324088699506544 121<br/>
nuclei_gbm_image08  :  0.8804250959880733 123<br/>
nuclei_lung_image06  :  0.9121998339236388 119<br/>
nuclei_lgg_image03  :  0.8737407376571477 117<br/>
nuclei_lgg_image05  :  0.8594481376923777 111<br/>
nuclei_lgg_image06  :  0.8453096210846751 113<br/>
nuclei_gbm_image06  :  0.8566324907627181 115<br/>
nuclei_gbm_image03  :  0.8456073644186511 118<br/>
nuclei_lung_image02  :  0.7613867304243873 117<br/>
nuclei_gbm_image07  :  0.8679014858538571 119<br/>
nuclei_lung_image03  :  0.8728274257561243 117<br/>
nuclei_hnsc_image01  :  0.8321472120607472 111<br/>
nuclei_hnsc_image05  :  0.8198542347323448 118<br/>
nuclei_lung_image07  :  0.9227191260561577 120<br/>
nuclei_hnsc_image06  :  0.8079416106090396 116<br/>
nuclei_lgg_image08  :  0.8623812006255764 121<br/>
nuclei_lung_image01  :  0.7257110540330373 112<br/>
nuclei_lgg_image01  :  0.8665360156781187 117<br/>
nuclei_hnsc_image02  :  0.8480114118805828 120<br/>
nuclei_hnsc_image07  :  0.841697827604559 119<br/>
nuclei_lgg_image07  :  0.8837092227144583 115<br/>
nuclei_lung_image04  :  0.8561029083091506 119<br/>
nuclei_hnsc_image08  :  0.8350518485518164 119<br/>
0.8494561127444595<br/>

# junk

In [ ]:
nuclei=imread('Results/prediction_CPM_17/nuclei_lung_image02.png')
boundary=imread('Results/prediction_CPM_17/bound_lung_image02.png')
gt=imread('Data/CPM_17/Test/NucleiMaps/lung_image02.png')
gt_bound=imread('Data/CPM_17/Test/BoundaryMaps/lung_image01.png')
# boundary=(boundary>60).astype(np.float32)

# # seg_image=watershed_seg(nuclei,boundary)

# nuclei=(nuclei>155).astype(np.float32)#threshold_otsu(nuclei)
# nuclei=ndimage.binary_fill_holes(nuclei).astype(int)

# # boundary=multiple_dialte(boundary,iter_count=2,selem=morphology.selem.square(5))
# # boundary=multiple_erosion(boundary,iter_count=1,selem=morphology.selem.square(5))

plt.figure(1,figsize=(10,10))
plt.imshow(nuclei)
plt.figure(2,figsize=(10,10))
plt.imshow(boundary)
plt.figure(3,figsize=(10,10))
plt.imshow(gt)
plt.figure(4,figsize=(10,10))
plt.imshow(gt_bound)

f = open('Data/CPM_17/Test/GT_Mask/lung_image02_mask.txt','r')
x = f.readlines()
n_array_gt=np.array([int(a) for a in x[1:]]).reshape(list(map(int, re.findall('\d+', x[0]))))
# n_array_gt=sort_n_array(n_array_gt)
labeled_gt=label(n_array_gt)
gt_regions=regionprops(labeled_gt)
plt.figure(5,figsize=(10,10))
plt.imshow(n_array_gt)
print("Number of regions",len(gt_regions))

In [ ]:

# print(np.amax(boundary_th))




plt.figure(7,figsize=(10,10))
plt.imshow(nuclei_seeds)


plt.figure(8,figsize=(10,10))
plt.imshow(nuclei_seeds)
labeled_seeds=label(nuclei_seeds)
seeds_regions=regionprops(labeled_seeds)
print(len(seeds_regions))


In [ ]:
nuclei_distance_map=ndi.distance_transform_edt(nuclei_seeds)
plt.figure(9,figsize=(10,10))
plt.imshow(1-nuclei_distance_map)

In [ ]:
local_maxi = peak_local_max(nuclei_distance_map, indices=False, footprint=np.ones((3, 3)),
                            labels=nuclei_th)
markers = ndi.label(local_maxi)[0]
labels = watershed(-nuclei_distance_map, markers, mask=nuclei_th)
plt.figure(10,figsize=(10,10))
plt.imshow(labels)

In [ ]:
get_fast_aji(sort_n_array(n_array_gt),sort_n_array(labels))

In [ ]:
nuclei_seeds=nuclei-boundary
nuclei_seeds[nuclei_seeds<0]=0
# nuclei_seeds=multiple_erosion(nuclei_seeds,4,selem=morphology.selem.square(3))

labeled_img=label(nuclei_seeds)
regions=regionprops(labeled_img)
print(len(regions))

# nuclei_seeds=ndimage.binary_fill_holes(nuclei_seeds).astype(int)
plt.imshow(nuclei_seeds)


In [ ]:

final_image=np.zeros_like(nuclei_seeds)
loop=tqdm(regions)
for i,region in enumerate(loop):


    coordinates=coord2array(list(region.coords))
    x_temp=np.zeros_like(nuclei_seeds)
    x_temp[coordinates]=1
    x_temp=skimage.morphology.reconstruction(x_temp, nuclei, method='dilation', selem=None, offset=None)



    final_image[np.where(x_temp==1)]=i+1
    final_image=final_image.astype(np.int16)
plt.figure(figsize=(10,10))   
plt.imshow(final_image)

In [ ]:
get_fast_aji(sort_n_array(n_array_gt),sort_n_array(final_image))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max


# Generate an initial image with two overlapping circles
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
image = np.logical_or(mask_circle1, mask_circle2)

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=image)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
nuclei_2=(nuclei>threshold_otsu(nuclei)).astype(np.float32)#threshold_otsu(nuclei)
boundary=(boundary>threshold_otsu(boundary)).astype(np.float32)
nuclei_seeds=nuclei_2-boundary
nuclei_seeds[nuclei_seeds<0]=0
nuclei_seeds=morphology.binary_opening(nuclei_seeds)#multiple_erosion(nuclei_seeds,2,selem=morphology.selem.square(3))
nuclei_seeds=ndimage.binary_fill_holes(nuclei_seeds).astype(int)
nuclei_seeds=ndi.label(nuclei_seeds)[0]

plt.imshow(nuclei_seeds)

In [ ]:
th=watershed(255-nuclei, nuclei_seeds, mask=ndimage.binary_fill_holes(nuclei_2).astype(int))

plt.imshow(th)

In [ ]:
get_fast_aji(sort_n_array(n_array_gt),sort_n_array(th))

In [ ]:
plt.imshow(ndi.distance_transform_edt(n_array_gt))

In [ ]:
labeled_img=label(n_array_gt)
regions=regionprops(labeled_img)
print(len(regions))
final_image=np.zeros_like(n_array_gt)
temp=np.zeros_like(n_array_gt)
# loop=tqdm(regions)
coordinates=coord2array(list(regions[23].coords))
final_image[coordinates]=1


# for i,region in enumerate(loop):


#     coordinates=coord2array(list(region.coords))
#     x_temp=np.zeros_like(nuclei_seeds)
#     x_temp[coordinates]=1
#     x_temp=ndi.distance_transform_edt(x_temp)
    



#     final_image[np.where(x_temp==1)]=i+1
#     final_image=final_image.astype(np.int16)
plt.figure(figsize=(10,10))   
plt.imshow(1-ndi.distance_transform_edt(final_image))

In [ ]:
x=np.reshape(ndi.distance_transform_edt(final_image),(-1,1))
for i in tqdm(range(len(x))):
    x[i]=(x[i]-np.min(x))/np.max(x)
    
plt.hist(x)

In [ ]:
temp=ndi.distance_transform_edt(final_image)
temp[temp>0]=1
plt.figure(figsize=(10,10)) 
plt.imshow(temp)

In [ ]:
plt.figure(figsize=(10,10)) 
plt.imshow((final_image))

In [ ]:
np.unique(ndi.distance_transform_edt(final_image)/255)

In [ ]:
distance_image=np.zeros_like(n_array,dtype=np.float64)
for i in tqdm(np.unique(n_array)):
    img_temp=np.zeros_like(n_array)
    img_temp[np.where(n_array==i)]=1
    distance = ndi.distance_transform_edt(img_temp)
    distance_image+=distance
plt.imshow(1-distance_image)
    

In [ ]:
bound_pred=imread('prediction_dual_2_CPM_17/bound_gbm_image01.png')
distance_bound = ndi.distance_transform_edt(bound_pred)
plt.imshow(distance_bound)

In [ ]:

for img in tqdm([x for x in os.listdir('prediction_CPM_17/') if 'proc' in x]):
    mask_name='_'.join(img.split('_')[1:-1])+'_mask.txt'
    pred_n_array=imread('prediction_CPM_17/'+img)
    
    f = open('other_data/CPM_17/Test/GT_Mask/'+mask_name, 'r')
    x = f.readlines()
    gt_n_array=np.array([int(a) for a in x[1:]]).reshape(list(map(int, re.findall('\d+', x[0]))))
    
    gt_threshold=np.zeros((*gt_n_array.shape,3))
    pred_threshold=np.zeros((*pred_n_array.shape,3))

    gt_threshold[np.where(gt_n_array>0)[0],np.where(gt_n_array>0)[1],1]=255
    pred_threshold[np.where(pred_n_array>0)[0],np.where(pred_n_array>0)[1],0]=255
    
    imsave('overlap/'+'_'.join(img.split('_')[1:-1])+'_pred.png',pred_threshold.astype(np.uint8))
    imsave('overlap/'+'_'.join(img.split('_')[1:-1])+'_gt.png',gt_threshold.astype(np.uint8))
    imsave('overlap/'+'_'.join(img.split('_')[1:-1])+'_overlap.png',(gt_threshold+pred_threshold).astype(np.uint8))
    

In [ ]:
f = open('Data/CPM_17/Test/GT_Mask/gbm_image01_mask.txt', 'r')
x = f.readlines()
n_array=np.array([int(a) for a in x[1:]]).reshape(list(map(int, re.findall('\d+', x[0]))))
plt.imshow(n_array)

In [ ]:
label_img = label(n_array)
regions = regionprops(label_img)
# print(len(np.unique(n_array)),len(regions))

In [ ]:
x_temp_img=np.zeros_like(n_array,dtype=np.float32)
y_temp_img=np.zeros_like(n_array,dtype=np.float32)
for i,region in enumerate(tqdm(regions)):
    coordinates=coord2array(list(region.coords))
    distance_x=(coordinates[0]-np.ones(len(coordinates[0]))*round(region.centroid[0]))
    distance_y=(coordinates[1]-np.ones(len(coordinates[1]))*round(region.centroid[1]))
    distance_x=((distance_x-np.min(distance_x))/(np.max(distance_x)-np.min(distance_x)))*(2) -1
    distance_y=((distance_y-np.min(distance_y))/(np.max(distance_y)-np.min(distance_y)))*(2) -1
#     distance_x=distance_x
    x_temp_img[coordinates]=distance_x
    y_temp_img[coordinates]=distance_y    

In [ ]:
print(x_temp_img[:200,:200])

In [ ]:
# x_temp=(x_temp_img+1)/2
print(np.max(x_temp_img),np.min(x_temp_img))
plt.figure(1,figsize=(10,10))
plt.imshow(x_temp_img)

In [ ]:
pred_dir_CoNSep='prediction_dual_2_CoNSep'
img_list=[x for x in os.listdir(pred_dir_CoNSep) if 'test' and 'nuc' in x and 'proc' not in x]
for img_name in tqdm(img_list):
    boundary=imread(pred_dir_CoNSep+'/'+'bound_'+'_'.join(img_name.split('_')[1:]))
    boundary=boundary>0.45*255
    nuclei=imread(pred_dir_CoNSep+'/'+img_name)
    nuclei=nuclei>0.45*255
    gt=np.load('other_data/CoNSeP/Test/Labels/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.npy')[:,:,0]
    overlap_imape=np.zeros((*nuclei.shape,3))
    gt_threshold=gt>0
    overlap_imape[np.where(boundary>0)[0],np.where(boundary>0)[1],0]=255
    overlap_imape[np.where(nuclei>0)[0],np.where(nuclei>0)[1],1]=255 
    overlap_imape[np.where(gt_threshold>0)[0],np.where(gt_threshold>0)[1],2]=255 
    
    imsave('overlap/'+img_name.split('.')[0]+'_gt.png',gt_threshold.astype(np.uint8)*255)
    imsave('overlap/'+img_name.split('.')[0]+'_overlap.png',(overlap_imape).astype(np.uint8))

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import numpy as np

In [ ]:
img=imread('Data/CPM_17/Test/H_gray/gbm_image01.png')
# img=np.zeros((100,100))
# # img[:50,50:]=1
# # img[50:,:50]=0.5
plt.imshow(img);plt.show()
img=np.expand_dims(img,axis=2).transpose(2,0,1)
img=torch.from_numpy(np.expand_dims(img,axis=0)).type(torch.FloatTensor)
img2=F.interpolate(img, size=(32,32))
img3=F.interpolate(img2, size=(500,500))
print(tuple(img2.size()[-2:]),img.size())

plt.imshow(np.squeeze(np.squeeze(img2.numpy(),0).transpose(1,2,0),axis=2));plt.show()
plt.imshow(np.squeeze(np.squeeze(img3.numpy(),0).transpose(1,2,0),axis=2));plt.show()

In [ ]:
!ls 
